## Barbara Karakyriakou
## MCB112 - Fall 2018
## homework 01:
## the case of the dead sand mouse

### 1. check that the gene names match 

In principle, the gene names in the two data files ought to match, but (sigh) you know that all kinds of things can go wrong in practice: people use different names for the same gene, spell them with different capitalization or punctuation or added spaces, and so on. <br>Write a Python script to compare the gene names in the two data files. Output the names that appear in Moriarty_SuppTable1 but not Adler_SuppTable2, if any. <br>You're especially suspicious of the Moriarty et al. data table because he mentions in his paper that he exported the supplementary methods tables from Microsoft Excel, and you've run across people on Twitter discussing not just one but two terrifying papers about how Excel has systematically corrupted the supplementary data files in many published articles. <br> If there's a difference - why?

In [1]:
# create a dictionary from the Moriarty text file w/the gene names as keys and the measured data as values
moriarty_data = {}                       
for line in open('Moriarty_SuppTable1'):
    if line[0] == '#': continue # skip header     
    line   = line.rstrip('\n') # remove the endings    
    fields = line.split() # split each row into fields with space delimiter        

    moriarty_data[fields[0]] = [float(s) for s in fields[1:6]]

In [2]:
# create a dictionary from the Adler text file w/the gene names as keys and the measured data as values
adler_data = {}                       
for line in open('Adler_SuppTable2'):
    if line[0] == '#': continue     
    line   = line.rstrip('\n')      
    fields = line.split()           

    adler_data[fields[0]] = [float(s) for s in fields[1:3]]

In [3]:
# list comprehension to find genes that are different in moriarty data
diff1 = [gene for gene in moriarty_data if gene not in adler_data]
# list comprehension to find genes that are different in adler data
diff2 = [gene for gene in adler_data if gene not in moriarty_data]
# combine the two lists
diff = diff1+diff2
print('The unique genes for each table are:', diff)

The unique genes for each table are: ['15-Sep', '2-Mar', '1-Mar', '10-Sep', '7-Mar', '4-Mar', '2-Sep', '11-Sep', '6-Mar', '11-Mar', '3-Mar', '8-Sep', '7-Sep', '14-Sep', '6-Sep', '1-Dec', '8-Mar', '5-Mar', '9-Mar', '12-Sep', '1-Sep', '4-Sep', '10-Mar', '9-Sep', '5-Sep', '3-Sep', 'MARCH4', 'MARCH1', 'SEPT10', 'MARCH8', 'SEP15', 'MARCH2', 'SEPT14', 'MARCH6', 'SEPT5', 'SEPT9', 'MARCH11', 'SEPT3', 'SEPT1', 'MARCH9', 'MARCH3', 'MARC2', 'SEPT8', 'MARCH7', 'SEPT7', 'MARCH5', 'MARC1', 'SEPT4', 'SEPT11', 'MARCH10', 'SEPT6', 'DEC1', 'SEPT12', 'SEPT2']


#### There are differences in both files. It appears that some gene names erroneously converted to dates.

### 2. explore the data

#### Write Python code to:

#### output the five genes with the highest mRNA synthesis rate. (i.e. in Adler_SuppTable2)

In [4]:
sorted(adler_data.items(), key=lambda kv: kv[1], reverse=True)[:5]

[('LINC00176', [118.6, 13.8]),
 ('MAP3K9', [102.8, 14.2]),
 ('FTSJ1', [99.5, 15.6]),
 ('PIGP', [83.3, 17.7]),
 ('GDAP2', [73.9, 9.4])]

In [ ]:
adler_data.items(

#### output the five genes with the longest mRNA halflife. (i.e. in Adler_SuppTable2)


In [27]:
sorted(adler_data.items(), key=lambda kv: kv[1][1], reverse=True)[:5]

[('BEX2', [3.6, 88.7]),
 ('LIPH', [7.9, 63.4]),
 ('NLK', [4.1, 62.9]),
 ('U2AF1', [9.6, 56.4]),
 ('TMEM186', [21.7, 53.9])]

#### output the five genes that have the highest ratio of expression at t=96 hours post-mortem vs. t=0 (i.e. in Moriarty_SuppTable1)

In [50]:
# create a dictionary with the ratio of expression t=96/t=0
with open('Moriarty_SuppTable1') as f1:
    ratio       = {}     
    for line in f1:
        if line[0] == '#': continue      
        line   = line.rstrip('\n')       
        fields = line.split()            # field [1] = 0hr. [5] = 96hr.

        ratio[fields[0]] = float(fields[5]) / float(fields[1])

In [52]:
# output the top five ratios
sorted(ratio.items(), key=lambda kv: kv[1], reverse=True)[:5]

[('BEX2', 39.559064682660185),
 ('LIPH', 28.366606982990152),
 ('NLK', 27.083050207625522),
 ('U2AF1', 20.402556469970232),
 ('TMEM186', 19.621011673151752)]

### 3. figure out what happened

This is partly an exercise in manipulating data files line-by-line in Python, but it's also designed to give you a Very Large Clue as to what happened in the dead sand mouse experiment.

Write a Python script that merges the two data files, line by line, merging them on gene name. That is, for each line in file 1 for gene X, find the corresponding line for gene X in file 2; we're going to write a single output file with one line per gene. The genes are in different orders in the files, so this merge isn't entirely trivial. For any gene name X that isn't found in both files (cough cough Excel corruption cough cough) just skip it. For each gene name that is found in both files, output one whitespace-delimited, column-justified data line consisting of 7 fields per line:

gene name
Four expression ratios relative to t=0: i.e. tpm[12h]/tpm[0], tpm[24h]/tpm[0], tpm[48h]/tpm[0], tpm[96h]/tpm[0], by processing the TPM data in Moriarty_SuppTable1
DNA synthesis rate (in mRNA/hr) and mRNA decay halflife (in hr) from Adler_SuppTable2
Save your merged dataset to a file.

Merging data sets, even crudely like this, is often useful as a step towards exploring data and looking for problems, outliers, and correlations.

Explore the data, however you want, for example by looking at the genes with the highest expression ratio t=96/t=0. What do you think is the real explanation for what happened in the dead sand mouse experiment?

In [7]:
import pandas as pd

# convert the moriarty data to a dataframe and display the first 10 rows
moriarty_df = pd.read_csv('Moriarty_SuppTable1', delim_whitespace=True, escapechar="#")
moriarty_df.head(10)

Matplotlib is building the font cache using fc-list. This may take a moment.


,gene_name,0h,12h,24h,48h,96h
0,anise,71.1,36.5,15.6,3.1,0.1
1,apricot,28.2,31.0,36.1,27.6,11.9
2,artichoke,75.3,73.2,61.0,42.4,9.7
3,arugula,50.7,46.3,43.5,25.3,4.7
4,asparagus,17.3,4.5,1.1,0.1,0.0
5,avocado,26.2,20.8,17.2,7.5,0.8
6,banana,11.1,10.2,9.1,4.5,0.7
7,basil,46.3,48.0,44.2,28.0,8.9
8,beet,47.8,44.8,33.6,21.0,4.4
9,blackberry,16.5,1.3,0.1,0.0,0.0


In [8]:
# convert the adler data to a dataframe and display the first 10 rows
adler_df = pd.read_csv('Adler_SuppTable2', delim_whitespace=True, escapechar="#")
adler_df.head(10)

,gene_name,synth_rate,halflife
0,ZFY,7.2,24.3
1,FAM222B,0.5,15.4
2,MRPS33,0.9,25.4
3,LSM1,2.5,17.4
4,TIAL1,31.2,6.8
5,USP53,12.4,21.8
6,RP11-302M6.4,0.3,13.6
7,COLEC10,1.9,10.1
8,MTFR1L,1.5,9.9
9,SIPA1L1,8.0,12.6


In [9]:
# merge the moriarty and adler dataframes ignoring the different gene names & display the first 10 rows
merged_df = moriarty_df.merge(adler_df)
merged_df.head(10)

,gene_name,0h,12h,24h,48h,96h,synth_rate,halflife
0,anise,71.1,36.5,15.6,3.1,0.1,8.2,8.0
1,apricot,28.2,31.0,36.1,27.6,11.9,1.7,17.1
2,artichoke,75.3,73.2,61.0,42.4,9.7,4.9,14.3
3,arugula,50.7,46.3,43.5,25.3,4.7,3.6,13.7
4,asparagus,17.3,4.5,1.1,0.1,0.0,3.1,5.6
5,avocado,26.2,20.8,17.2,7.5,0.8,2.3,11.6
6,banana,11.1,10.2,9.1,4.5,0.7,0.9,12.7
7,basil,46.3,48.0,44.2,28.0,8.9,2.9,15.1
8,beet,47.8,44.8,33.6,21.0,4.4,3.4,13.3
9,blackberry,16.5,1.3,0.1,0.0,0.0,3.9,3.7


In [10]:
# store data in a csv file at the current directory 
merged_df.to_csv('merged_data.csv', index = False, header=True)

In [11]:
# explore data sorting by the genes with the highest t=96/t=0 expression ratio
(merged_df.sort_values(by=['96h'], ascending=False)).head(10)

,gene_name,0h,12h,24h,48h,96h,synth_rate,halflife
15109,TMEM186,1285.0,2154.7,4079.0,8581.4,25213.0,21.7,53.9
12958,CAND1,1039.7,1815.3,2615.8,5888.8,13543.7,21.4,45.3
8527,BEX2,329.3,600.2,1292.2,3126.9,13026.8,3.6,88.7
4380,LIPH,446.8,1071.2,1748.4,3692.4,12674.2,7.9,63.4
2867,GTDC1,960.3,1724.1,2769.7,5484.0,12031.2,21.1,44.2
19903,U2AF1,571.1,1015.0,1810.9,3913.0,11651.9,9.6,56.4
5267,TRIO,1079.2,1753.1,2384.1,4889.0,8731.7,28.1,36.5
4346,LAMP3,1564.1,2399.7,3202.6,5083.1,8526.4,48.8,30.9
218,SLC25A33,735.6,1167.7,1746.3,3131.9,8204.5,16.7,40.2
16124,NLK,264.9,492.4,945.4,2000.6,7174.3,4.1,62.9


In [12]:
# explore data sorting by the genes with the highest halflife
(merged_df.sort_values(by=['halflife'], ascending=False)).head(10)

,gene_name,0h,12h,24h,48h,96h,synth_rate,halflife
8527,BEX2,329.3,600.2,1292.2,3126.9,13026.8,3.6,88.7
4380,LIPH,446.8,1071.2,1748.4,3692.4,12674.2,7.9,63.4
16124,NLK,264.9,492.4,945.4,2000.6,7174.3,4.1,62.9
19903,U2AF1,571.1,1015.0,1810.9,3913.0,11651.9,9.6,56.4
15109,TMEM186,1285.0,2154.7,4079.0,8581.4,25213.0,21.7,53.9
16598,CHAD,105.0,202.1,339.5,695.6,2037.5,1.9,53.5
17923,LRG1,41.6,83.4,132.5,303.9,752.8,0.8,52.0
6302,OR14J1,141.9,267.5,473.9,984.4,2521.7,2.9,50.4
1273,CDC42SE1,33.6,65.6,103.4,230.5,593.4,0.6,49.6
13269,TMEM120B,82.1,166.3,253.7,524.2,1440.8,1.7,48.7


In [13]:
# explore data sorting by the genes with the highest synthesis rate
(merged_df.sort_values(by=['synth_rate'], ascending=False)).head(10)

,gene_name,0h,12h,24h,48h,96h,synth_rate,halflife
17764,LINC00176,1615.0,1558.6,1488.1,861.4,185.1,118.6,13.8
14067,MAP3K9,1489.0,1509.3,1412.0,963.7,217.7,102.8,14.2
8247,FTSJ1,1582.1,1794.5,1553.1,1141.6,417.8,99.5,15.6
19860,PIGP,1501.1,1865.1,1933.2,1678.4,801.9,83.3,17.7
1157,GDAP2,685.8,477.2,280.2,70.4,3.1,73.9,9.4
14033,ATP6V1D,1655.8,2452.9,3396.0,4211.4,5210.0,64.4,26.1
9715,SIT1,439.7,174.6,67.4,6.6,0.0,64.1,6.9
1006,PTBP2,802.3,845.4,740.3,450.6,100.2,60.2,13.8
12209,ADRB1,575.7,381.3,242.8,60.1,3.1,59.5,9.5
17397,CST1,1347.8,1604.5,2297.3,2323.6,1991.0,58.5,22.1


#### *I will filter the data to check each gene of the experiment diagrams.*

In [14]:
# filter data by gene name
merged_df[merged_df.values  == ('APOBEC3G')]

,gene_name,0h,12h,24h,48h,96h,synth_rate,halflife
19586,APOBEC3G,142.8,198.9,230.9,204.5,112.2,8.4,18.8


* 'APOBEC3G' appears like it continues being expressed after the mouse is dead, but in reality this is due to slow decay of the mRNA. RNA-seq measures the relative abundance of each transcript, not the absolute abundance, so it looks like the "expression" of the more stable mRNAs is going up, just because they're going away slower. 

In [15]:
merged_df[merged_df.values  == ('cilantro')]

,gene_name,0h,12h,24h,48h,96h,synth_rate,halflife
24,cilantro,394.3,294.7,184.3,62.2,3.7,37.7,10.3


* 'cilantro's' expression drops immediately after the mouse dies, which means that this mRNA is not stable.

In [16]:
merged_df[merged_df.values  == ('GRIN2C')]

,gene_name,0h,12h,24h,48h,96h,synth_rate,halflife
16781,GRIN2C,210.6,260.8,306.0,401.3,233.7,9.8,20.8


* 'GRIN2C' similar to 'APOBEC3G' appears as being expressed due to existing mRNA that decays slowly after the mouse is dead. An mRNA with a moderate halflife will go up (as the least stable mRNAs disappear quickly from the population), then go down (as the moderate-halflife mRNA decays faster than more stable mRNAs).

In [17]:
merged_df[merged_df.values  == ('TMEM120B')]

,gene_name,0h,12h,24h,48h,96h,synth_rate,halflife
13269,TMEM120B,82.1,166.3,253.7,524.2,1440.8,1.7,48.7


* 'TMEM120B' is also similar to 'GRIN2C' similar to 'APOBEC3G. Overall, we see that the least stable mRNAs go away fast, while the more stable mRNAs decay in a slower rate. mRNAs with higher halflife appear as being expressed after death, but this is not true. As noted above, RNA-seq measures the relative abundance of each transcript, not the absolute abundance, so it looks like the "expression" of the more stable mRNAs is going up,